In [35]:
%run 2gis_functions.ipynb

Ready: 2023-03-25 07:38:21


In [2]:
findspark.init()
from pyspark.sql import SparkSession
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder\
        .master("local[*]")\
        .appName('PySpark_Tutorial')\
        .getOrCreate()

# Москва запросы и телефоны

In [5]:
rubrics = pd.read_excel('files/final_svod_master.xlsx')[['Рубрика']].iloc[:70]

df_all = dd.read_parquet('files/df_all')
df_filtered = df_all.merge(rubrics, left_on='rubricName', right_on='Рубрика')

df_filtered.to_parquet('df_filtered')

df_filtered = dd.read_parquet('df_filtered')

In [5]:
df = spark.read.parquet("df_filtered")#.withColumn('new_tx',normalize_word(coalesce(col('tx'), lit(''))))

df.createOrReplaceTempView('df')

grouped_df = spark.sql('''

select
    rubricName
    ,year
    ,tx
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year(date) year
        ,lower(tx) tx
        ,count(*) cnt
    from df
    group by 
        rubricName
        ,year(date)
        ,lower(tx)
        ) t
''')#.where('rn < 16')

In [6]:
%%time
grouped_df.write.mode('overwrite').parquet('df_tx_grouped')

CPU times: total: 78.1 ms
Wall time: 3min 34s


In [41]:
%%time
spark.read.parquet("df_tx_grouped").where('tx is not null and year in (2020,2021,2022,2023)').write.mode('overwrite').parquet('df_tx_grouped_filtered')

CPU times: total: 0 ns
Wall time: 4 s


# Расчет

In [104]:
import string
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from nltk.corpus import stopwords
russian_stopwords = stopwords.words("russian")

def normalize_word(word):
    return (sorted([morph.normal_forms(str(i))[0].strip(string.punctuation + string.whitespace) for i in word.split(' ')]))

normalize_word_udf = udf(normalize_word, ArrayType(StringType()))

def check_stopwords(word, russian_stopwords = russian_stopwords):
    if word not in russian_stopwords and len(word)>1:
        for letter in word:
            if letter in string.punctuation or letter in string.digits:
                return 0
        return 1
    return 0

check_stopwords_udf = udf(check_stopwords, StringType())

In [4]:
df_tx_grouped_normalize = (spark.read.parquet("df_tx_grouped_filtered")#.limit(3000)
                           .withColumn('new_tx',normalize_word_udf(col('tx')))
                    )

In [5]:
%%time
df_tx_grouped_normalize.write.mode('overwrite').parquet('df_tx_grouped_normalize')

CPU times: total: 656 ms
Wall time: 27min 22s


In [16]:
df_tx_grouped_normalize_agg = (
    spark.read.parquet(r"df_tx_grouped_normalize")
    .withColumn("new_tx_ex", f.explode(f.col("new_tx")))
    .where('coalesce(new_tx_ex, "") !="" ')
    .groupBy(f.col("rubricName"),f.col("new_tx_ex"),f.col("year")) 
    .agg(f.sum("cnt").alias("count"))
    .sort(desc('count'))
    #.show()
)

In [17]:
%%time
df_tx_grouped_normalize_agg.write.mode('overwrite').parquet('df_tx_grouped_normalize_agg')

CPU times: total: 0 ns
Wall time: 6.88 s


In [105]:
df_tx = spark.read.parquet(r"df_tx_grouped_normalize_agg").withColumn('check', check_stopwords_udf(f.col('new_tx_ex'))).where('check=1')

In [106]:
df_tx.createOrReplaceTempView('df_tx')

In [112]:
df_tx_grouped_normalize_final = spark.sql('''
with t as
    (
    select
        rubricName
        ,new_tx_ex
        ,case when year=2020 then sum(count) else 0 end cnt_2020
        ,case when year=2021 then sum(count) else 0 end cnt_2021
        ,case when year=2022 then sum(count) else 0 end cnt_2022
        ,case when year=2023 then sum(count) else 0 end cnt_2023
    from df_tx
    group by 
        rubricName
        ,new_tx_ex
        ,year
    )
, t1 as (        
select
    rubricName
    ,new_tx_ex
    ,max(cnt_2020) cnt_2020
    ,max(cnt_2021) cnt_2021
    ,max(cnt_2022) cnt_2022
    ,max(cnt_2023) cnt_2023
from t
group by new_tx_ex, rubricName
)

select *, cnt_2020+cnt_2021+cnt_2022+cnt_2023 as cnt_total, row_number() over(partition by rubricName order by cnt_2020+cnt_2021+cnt_2022+cnt_2023 desc) rn
from t1
        
''').where('rn < 31')

df_tx_grouped_normalize_final = spark.sql('''

select
    rubricName
    ,year
    ,new_tx_ex
    ,cnt
    ,row_number() over(partition by rubricName ,year order by cnt desc) rn
from (
    select
        rubricName
        ,year
        ,new_tx_ex
        ,sum(count) cnt
    from df_tx
    group by 
        rubricName
        ,year
        ,new_tx_ex
        ) t
''').where('rn < 16')

In [113]:
df_tx_grouped_normalize_final.write.mode('overwrite').parquet('df_tx_grouped_normalize_final')

In [114]:
spark.read.parquet('df_tx_grouped_normalize_final').count()

2100

In [115]:
pdf = spark.read.parquet('df_tx_grouped_normalize_final').toPandas()

In [121]:
cols = ['Рубрика', 'Поисковое слово', 'Количество запросов 2020', 'Количество запросов 2021', 'Количество запросов 2022', 'Количество запросов 2023', 'Количество запросов, итого', 'Рейтинг по запросам, итого']

In [122]:
pdf.columns = cols

In [123]:
pdf

,Рубрика,Поисковое слово,Количество запросов 2020,Количество запросов 2021,Количество запросов 2022,Количество запросов 2023,"Количество запросов, итого","Рейтинг по запросам, итого"
0,Авторемонт и техобслуживание (СТО),автосервис,56958,64519,62769,54880,239126,1
1,Авторемонт и техобслуживание (СТО),сто,10161,18892,22543,24166,75762,2
2,Авторемонт и техобслуживание (СТО),техобслуживание,1101,6153,9638,10638,27530,3
3,Авторемонт и техобслуживание (СТО),авторемонт,1188,5757,9355,10660,26960,4
4,Авторемонт и техобслуживание (СТО),рядом,3420,6514,7832,8119,25885,5
5,Авторемонт и техобслуживание (СТО),сервис,3829,6116,6313,6871,23129,6
6,Авторемонт и техобслуживание (СТО),авто,3806,4604,4704,4706,17820,7
7,Авторемонт и техобслуживание (СТО),замена,3068,4282,3058,2092,12500,8
8,Авторемонт и техобслуживание (СТО),масло,3104,4213,3011,2008,12336,9
9,Авторемонт и техобслуживание (СТО),ремонт,2408,2759,2887,3381,11435,10


In [124]:
pdf.to_excel('df_tx_grouped_normalize_final.xlsx')